# Vertex AI Experiment Example

This notebook demonstrates how to use the `VertexAIExperiment` class to submit preprocessing and training jobs to Vertex AI.

In [2]:
import os
from pathlib import Path
import json
from typing import Dict
import sys

def setup_project_path():
    """Add project root to Python path by searching for .git directory"""
    current_path = Path.cwd()
    
    # Search up the directory tree for .git folder or pyproject.toml
    root_indicators = ['.git', 'pyproject.toml']
    
    while current_path != current_path.parent:
        if any((current_path / indicator).exists() for indicator in root_indicators):
            sys.path.append(str(current_path))
            return current_path
        current_path = current_path.parent
    
    raise RuntimeError(
        "Could not find project root. "
        "Please run this notebook from within the project directory."
    )

# Setup path
project_root = setup_project_path()
print(f"Project root detected at: {project_root}")

Project root detected at: /home/steffen/sign-language-translator


In [3]:
# Import the necessary classes
from models.vertex_ai.experiment import VertexAIExperiment, ExperimentConfig

Environment variables loaded from .env


## Initialize the Experiment

First, we initialize the `VertexAIExperiment` with the default configuration. This configuration is loaded from a YAML file.

In [4]:
# Initialize the experiment with default configuration
experiment = VertexAIExperiment()

## Submit a Preprocessing Job

Next, we submit a preprocessing job. This job will execute a Python script on Vertex AI. The script should be located in your project directory.

In [5]:
# Submit a preprocessing job
preprocessing_job = experiment.submit_preprocessing_job(
    script_path="path/to/preprocessing_script.py",
    args={"--input-bucket": "gs://your-input-bucket", "--output-bucket": "gs://your-output-bucket"}
)

print("Preprocessing job submitted.")

FileNotFoundError: Script not found at path/to/preprocessing_script.py

## Submit a Training Job

Finally, we submit a training job. This job will also execute a Python script on Vertex AI. The script should be compatible with the requirements specified in the `submit_training_job` method.

In [4]:
# Submit a training job
training_job = experiment.submit_training_job(
    script_path="path/to/training_script.py",
    args={"--train-data": "gs://your-output-bucket/train-data", "--model-dir": "gs://your-model-bucket"}
)

print("Training job submitted.")